In [1]:
:load ./learning-2022/files/LectureNotes/Sections/interpreter/Parsing

{-# LANGUAGE TypeSynonymInstances #-}
{-# LANGUAGE FlexibleInstances    #-}

### Syntax

In [2]:
import qualified Data.Map as Map

In [3]:
data Register = ACC
              | NIL
           -- non-addressable:
              | BAK
              | IP
              deriving (Show, Read, Eq)

data Operand = Regi Register
             | Cons Int
             | Port String
             | IN  -- also ports
             | OUT
             deriving (Read, Eq)

type Condition = Int -> Bool
type Label = String

data Operator = MOV Operand
              | SWP
              | ADD
              | SUB
              | NEG
              | JRO Condition
              | HCF
           -- runtime only:
              | Mov Operand
           -- parsing only
              | Jmp Condition Label
           -- replaced with equivalent:
           -- | NOP                         -> ADD NIL
           -- | SAV                         -> MOV ACC BAK
           -- | JMP | JEZ | JNZ | JGZ | JLZ -> JRO
              deriving (Show, Read, Eq)

data Operation = O
                 { operator :: Operator
                 , operand  :: Operand
                 } deriving (Read, Eq)

type State = Register -> Int

data Mode = Default
          | Write Int Operand
          deriving (Show, Eq)

data Node = N
            { lines :: [Operation]
            , state :: State 
            , mode  :: Mode
            } deriving Eq

data Program = P
               { nodes :: Map.Map String Node
               , ins   :: [Int]
               , outs  :: [Int]
               } deriving Eq


-- instances

instance Show Operand where
  show (Regi x) = show x
  show (Cons x) = show x
  show (Port x) = "@" ++ x
  show IN       = "IN"
  show OUT      = "OUT"

instance Show Operation where
  show (O (MOV o) x) = "MOV " ++ show x ++ " " ++ show o
  show (O (Mov o) x) = "Mov " ++ show x ++ " " ++ show o
  show (O x y) = show x ++ " " ++ show y

instance Show Condition where
  show c = case map c [(-1)..1] of
             [True, True, True ] -> "__"
             [False,True, False] -> "EZ"
             [True, False,True ] -> "NZ"
             [False,False,True ] -> "GZ"
             [True, False,False] -> "LZ"

instance Read Condition where
  readsPrec _ _ = []
  
instance Eq Condition where
  x == y = map x [(-1)..1] == map y [(-1)..1]

instance Eq State where
  x == y = map x [ACC, BAK, IP] == map y [ACC, BAK, IP]

instance Show Node where
  show = show . lines
  
instance Show Program where
  show (P ns is os) = show ns ++ show is ++ show os

### Parsing

In [4]:
import Text.Read
import Data.Maybe
import Control.Monad
import Data.Either

import Parsing

In [5]:
goodchar :: Parser Char
goodchar = sat (`notElem` " :#\n")

word :: Parser String
word = token $ many goodchar

name :: Parser String
name = token $ do char '@'
                  word

register :: Parser Register
register = do symbol "ACC"
              return  ACC
       <|> do symbol "NIL"
              return  NIL

pOperand :: Parser Operand
pOperand =  Regi <$> register
        <|> Cons <$> integer
        <|> Port <$> name
        <|> do symbol "IN"
               return  IN
        <|> do symbol "OUT"
               return  OUT    

operation :: Parser Operation
operation =  do symbol "MOV"
                o <- pOperand
                r <- pOperand
                return $ O (MOV r) o
         <|> do symbol "ADD"
                O ADD <$> pOperand
         <|> do symbol "SUB"
                O SUB <$> pOperand
         <|> do symbol "JRO"
                O (JRO (const True)) <$> pOperand
         <|> do symbol "JMP"
                w <- word
                return $ O (Jmp (const True) w) (Regi NIL)
         <|> do symbol "JEZ"
                w <- word
                return $ O (Jmp (==0)        w) (Regi NIL)
         <|> do symbol "JNZ"
                w <- word
                return $ O (Jmp (/=0)        w) (Regi NIL)
         <|> do symbol "JGZ"
                w <- word
                return $ O (Jmp (>0)         w) (Regi NIL)
         <|> do symbol "JLZ"
                w <- word
                return $ O (Jmp (<0)         w) (Regi NIL)
         <|> do symbol "SAV"
                return $ O (MOV $ Regi BAK) (Regi ACC)
         <|> do symbol "NOP" 
                return $ O ADD (Regi NIL)
         <|> do xs <- word
                guard $ isJust (readMaybe xs :: Maybe Operator)
                return $ O (read xs :: Operator) (Regi NIL)

                
label :: Parser Label
label = token $
          do w <- word
             char ':'
             return w

node :: Parser [Either Operation Label]
node = many $
        Left  <$> operation
    <|> Right <$> label

namedNode :: Parser (String, [Either Operation Label])
namedNode = do x <- name
               y <- node
               return (x, y) 

-- consume labels and replace them with relative offsets
process :: [Either Operation Label] -> (Label -> Int) -> [Operation] -> [Operation]
process [] f os = p os
  where p [] = []
        p xs = p (init xs) ++ [q (last xs)]
          where q (O (Jmp c s) _) = O (JRO c) (Cons (f s - length (init xs)))
                q x         = x
process (Left  o : es) f n = process es f (n ++ [o])
process (Right l : es) f n = process es f' n
  where f' l' | l' == l   = if all isRight es then 0 else length n
              | otherwise = f l'
           
program :: Parser [(String, [Either Operation Label])]
program = many namedNode

processProgram :: [(String, [Either Operation Label])] -> Map.Map String Node
processProgram = foldr (\ x -> Map.insert (fst x) (N (process (snd x) (const undefined) []) (const 0) Default)) Map.empty

parseProgram :: String -> Program
parseProgram x = P ((processProgram . fst . head . parse program) x) [] []

### Interpreting

In [6]:
-- clamp a value to a range
clamp :: Int -> Int -> Int -> Int
clamp l h x = max l $ min h x

-- change value of internal register
update :: Register -> Int -> Node -> Node
update NIL _ n = n
update r   x (N os s m) = N os s' m
  where s' r' | r' == r   = clamp (-999) 999 x
              | otherwise = s r'

-- check value of internal register
get :: Register -> Node -> Int
get r n = state n r

-- number of lines in a node
len :: Node -> Int
len = length . lines

-- current line of a node
current :: Node -> Operation
current (N os s Default)     = os !! s IP
current (N os _ (Write i d)) = O (Mov d) (Cons i)

-- set mode to writing or not writing
mode :: Mode -> Node -> Node
mode m (N os s _) = N os s m

-- safely set IP to next line
inc :: Node -> Node
inc n = mode Default $ update IP ((get IP n + 1) `mod` len n) n

-- safely set IP to any line
jump :: Int -> Node -> Node
jump i n = update IP (clamp 0 (len n-1) i) n

-- check if a node or input is being read by any node
checkAnyRead :: Operand -> Program -> Bool
checkAnyRead o p = any ((== o) . operand . current) (Map.elems $ nodes p)

-- check if a certain node or output is reading another certain node
checkRead :: Operand -> Operand -> Program -> Bool
checkRead (Port s) o (P ns _ _) = operand (current (ns Map.! s)) == o
checkRead OUT _ _ = True

-- gather all the values being written to a node or output
readAny :: Operand -> Program -> [Int]
readAny o p = [f $ operand $ current a | a <- Map.elems $ nodes p, operator (current a) == Mov o]
  where f (Cons x) = x

-- evaluate an operand into a constant
eval :: Operand -> Operand -> Node -> Program -> Maybe Int
eval (Cons x) _ _ _ = Just x
eval (Regi r) o n _ = Just $ get r n
eval  IN      _ _ (P _ (i:is) _) = Just i
eval  IN      _ _ (P _ []     _) = Nothing
eval (Port x) o _ (P ns _ _) = 
  case current (ns Map.! x) of
    O (Mov o) (Cons y) -> Just y
    _                  -> Nothing

-- internally execute a line of code
e :: Operator -> Int -> Node -> Node
e (MOV (Regi r)) x n = update r x $ inc n
e (MOV d) x n = mode (Write x d) n
e (Mov _) _ n = n
e  SWP    _ n = update BAK (get ACC n) $ update ACC (get BAK n) $ inc n
e  ADD    x n = update ACC (get ACC n + x) $ inc n
e  SUB    x n = update ACC (get ACC n - x) $ inc n
e  NEG    _ n = update ACC (negate $ get ACC n) $ inc n
e (JRO c) x n = if c $ get ACC n then jump (get IP n + x) n else inc n
e  HCF    _ _ = error "halted and caught fire"

-- process a program for 1 cycle
step :: Program -> Program
step p = P (Map.fromList [f n | n <- Map.toList $ nodes p]) 
           (if not (null $ ins p) && checkAnyRead IN p then tail $ ins p else ins p)
           (outs p ++ readAny OUT p)
  where f (k, v) = case ((operator . current) v, eval ((operand . current) v) (Port k) v p) of
          (Mov r, _)  -> if checkRead r (Port k) p then (k, inc v) else (k, v)
          (o, Just x) -> (k, e o x v)
          _           -> (k, v)

In [7]:
type HaltCondition = [Program] -> [Program]

input :: [Int] -> Program -> Program
input is (P ns _ os) = P ns is os

-- runs a program infinitely or until a condition is met. takes a continuation
run :: Program -> [Int] -> HaltCondition -> ([Program] -> a) -> a
run p is c f = f $ c $ iterate step $ input is p

-- program runs until all nodes are blocked
stuck :: HaltCondition
stuck = takeWhile (\p -> p /= step p)

timeout :: Int -> HaltCondition
timeout = take

def :: HaltCondition
def = stuck . timeout 10000

results :: [Program] -> [Int]
results = outs . last

### Visualisation

In [8]:
-- adapted from Data.List.HT

pR :: Int -> String -> String
pR n xs = take n $ xs ++ repeat ' '

pL :: Int -> String -> String
pL n xs = replicate (n - length xs) ' ' ++ xs

In [9]:
logNode :: Node -> String
logNode n = concat [
  pL  2 $ show $ get IP n,  ": ",
  pR 12 $ show $ current n, "|",
  pL  4 $ show $ get ACC n, "| (",
  pL  4 $ show $ get BAK n, ")       "]

log :: [Program] -> IO ()
log ps = putStr $ unlines [concatMap logNode (Map.elems $ nodes p) ++ show (ins p) ++ show (outs p) | p <- ps]

data VisState = X | Y | C

xMAX = 30
yMAX = 18

-- converts program output into a pixel matrix
vis :: [Program] -> [Int]
vis ps = v X 0 0 (results ps) (replicate (xMAX*yMAX) 0)
  where
    v :: VisState -> Int -> Int -> [Int] -> [Int] -> [Int]
    v _ _ _ [] i = i
    v s x y (a:as) i = case (s, x<xMAX && y<yMAX, a<0) of
      (_, _, True)  -> v X 0 0 as i
      (X, _, _)     -> v Y a 0 as i
      (Y, _, _)     -> v C x a as i
      (C, False, _) -> v C x y as i
      (C, True, _)  -> v C (x+1) y as $ take (xMAX*y+x) i ++ [a] ++ drop (xMAX*y+x+1) i

render :: [Int] -> String
render [] = []
render xs = concatMap ((++ "█") . f) (take xMAX xs) ++ "\n" ++ render (drop xMAX xs)
  where f x = case x of
                1 -> "\x1b[38;5;239m" -- dark grey
                2 -> "\x1b[38;5;247m" -- bright grey
                3 -> "\x1b[38;5;255m" -- white
                4 -> "\x1b[31m"       -- red
                _ -> "\x1b[38;5;16m"  -- black

image :: [Program] -> IO ()
image = putStr . render . vis

### Testing

In [10]:
makespeare = unlines
  ["@m          ",
   "A: ADD -143 ",
   "JLZ B       ",
   "ADD -197    ",
   "B: ADD 161  ",
   "MOV ACC OUT ",
   "SWP         ",
   "10: JLZ A   ",
   "MOV 3 OUT   ",
   "ADD 2       "]

m = parseProgram makespeare

run m [] (timeout 3597) image

██████████████████████████████
██████████████████████████████
██████████████████████████████
██████████████████████████████
██████████████████████████████
██████████████████████████████
██████████████████████████████
██████████████████████████████
██████████████████████████████
██████████████████████████████
██████████████████████████████
██████████████████████████████
██████████████████████████████
██████████████████████████████
██████████████████████████████
██████████████████████████████
██████████████████████████████
██████████████████████████████

In [11]:
stored = parseProgram $ unlines
  ["@1            ",
   "   MOV IN @2  ",
   "              ",
   "@2            ",
   "   MOV @1 ACC ",
   "   SWP        ",
   "   MOV @1 ACC ",
   "   SWP        ",
   "L: SWP        ",
   "   MOV ACC @3 ",
   "   SWP        ",
   "   SUB 1      ",
   "   JNZ L      ",
   "              ",
   "@3            ",
   "   MOV 0 OUT  ",
   "   MOV ACC OUT",
   "   SWP        ",
   "   ADD 30     ",
   "L: MOV @2 OUT ",
   "   SUB 1      ",
   "   JGZ L      ",
   "   MOV -1 OUT ",
   "   SWP        ",
   "   ADD 1      "]
   
run stored [36,2,26,2,23,2,34,3,37,2,30,3,40,1,36,3,40,3,39,2,21,0,26,1,38,1,22,3,34,1,20,2,42,2] def image

██████████████████████████████
██████████████████████████████
██████████████████████████████
██████████████████████████████
██████████████████████████████
██████████████████████████████
██████████████████████████████
██████████████████████████████
██████████████████████████████
██████████████████████████████
██████████████████████████████
██████████████████████████████
██████████████████████████████
██████████████████████████████
██████████████████████████████
██████████████████████████████
██████████████████████████████
██████████████████████████████

In [12]:
b = parseProgram $ unlines
  ["@1            ",
   "   MOV 0 ACC  ",
   "   SUB IN     ",
   "&: ADD @2     ",
   "   MOV -2 @2  ",
   "   JLZ &      ",
   "   MOV @2 @2  ",
   "   MOV ACC @2 ",
   "              ",
   "@2            ",
   "&: MOV ACC @1 ",
   "   SUB @1     ",
   "   JGZ &      ",
   "   MOV @1 OUT "]

run b [1,4] def log

run b (map (^2) [1..31]) def length

 0: MOV 0 ACC   |   0| (   0)        0: MOV ACC @1  |   0| (   0)       [1,4][]
 1: SUB IN      |   0| (   0)        0: Mov 0 @1    |   0| (   0)       [1,4][]
 2: ADD @2      |  -1| (   0)        0: Mov 0 @1    |   0| (   0)       [4][]
 3: MOV -2 @2   |  -1| (   0)        1: SUB @1      |   0| (   0)       [4][]
 3: Mov -2 @2   |  -1| (   0)        1: SUB @1      |   0| (   0)       [4][]
 4: JRO LZ -2   |  -1| (   0)        2: JRO GZ -2   |   2| (   0)       [4][]
 2: ADD @2      |  -1| (   0)        0: MOV ACC @1  |   2| (   0)       [4][]
 2: ADD @2      |  -1| (   0)        0: Mov 2 @1    |   2| (   0)       [4][]
 3: MOV -2 @2   |   1| (   0)        1: SUB @1      |   2| (   0)       [4][]
 3: Mov -2 @2   |   1| (   0)        1: SUB @1      |   2| (   0)       [4][]
 4: JRO LZ -2   |   1| (   0)        2: JRO GZ -2   |   4| (   0)       [4][]
 5: MOV @2 @2   |   1| (   0)        0: MOV ACC @1  |   4| (   0)       [4][]
 5: MOV @2 @2   |   1| (   0)        0: Mov 4 @1    |   4| (

2823

In [13]:
example1 = parseProgram $ unlines [
  "@powerof2    ",
  "  MOV IN  ACC",
  "  SAV        ",
  "  MOV 1   ACC",
  "             ",
  "L:SWP        ",
  "  JEZ E      ",
  "  JLZ F      ",
  "  SUB 1      ",
  "  SWP        ",
  "  ADD ACC    ",
  "  JMP L      ",
  "             ",
  "F:HCF        ",
  "             ",
  "E:SWP        ",
  "  MOV ACC OUT"]

run example1 [0..11] def results

[1,2,4,8,16,32,64,128,256,512,999]